# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,72.00,33,75,5.75,US,1685051598
1,1,puerto natales,-51.7236,-72.4875,32.59,91,98,5.48,CL,1685051598
2,2,yellowknife,62.4560,-114.3525,57.72,54,40,9.22,CA,1685051598
3,3,waitangi,-43.9535,-176.5597,57.81,85,7,4.00,NZ,1685051598
4,4,roma,43.2128,-75.4557,63.52,47,0,7.00,US,1685051396


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from citipy import citipy

# Import API key
from api_keys import geoapify_key

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Get unique city coordinates
unique_cities = city_data_df[['Lat', 'Lng']].drop_duplicates()

# Create empty columns for city names and countries
unique_cities['City'] = ""
unique_cities['Country'] = ""

# Loop through each row and find the nearest city
for index, row in unique_cities.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Use citipy to find the nearest city
    city = citipy.nearest_city(latitude, longitude)
    
    # Store the city name and country code in the DataFrame
    unique_cities.at[index, 'City'] = city.city_name
    unique_cities.at[index, 'Country'] = city.country_code
    
# Merge the city names and countries back to the main DataFrame
city_data_df = pd.merge(city_data_df, unique_cities[['Lat', 'Lng', 'City', 'Country']], on=['Lat', 'Lng'])

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='Blues',
    size='Humidity',
    title='City Humidity',
    hover_cols=['City', 'Humidity'],
    geo=True,
    tiles='CartoLight'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Specify your ideal weather conditions
max_temperature = 72
min_temperature = 21
max_wind_speed = 4.5
cloudiness = 0

# Filter the DataFrame based on the weather conditions
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] < max_temperature) &
    (city_data_df['Max Temp'] > min_temperature) &
    (city_data_df['Wind Speed'] < max_wind_speed) &
    (city_data_df['Cloudiness'] == cloudiness)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display the sample data
ideal_cities_df.head()

,City_ID,City_x,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country_x,Date,City_y,Country_y
60,60,nsanje,-16.9200,35.2620,65.88,87,0,1.72,MW,1685051611,nsanje,mw
79,79,belo sur tsiribihina,-19.7000,44.5500,62.58,80,0,2.33,MG,1685051615,belo sur tsiribihina,mg
163,163,stanley,54.8680,-1.6985,55.89,78,0,1.14,GB,1685051633,stanley,gb
165,165,estrela,-28.0611,-50.9383,57.43,82,0,3.89,BR,1685051634,estrela,br
277,277,kafr takharim,36.1168,36.5153,63.28,83,0,3.69,SY,1685051657,kafr takharim,sy


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Create a copy of the city_data_df DataFrame
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City_x,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country_x,Date,City_y,Country_y,Hotel Name
0,0,badger,64.8000,-147.5333,72.00,33,75,5.75,US,1685051598,badger,us,
1,1,puerto natales,-51.7236,-72.4875,32.59,91,98,5.48,CL,1685051598,puerto natales,cl,
2,2,yellowknife,62.4560,-114.3525,57.72,54,40,9.22,CA,1685051598,yellowknife,ca,
3,3,waitangi,-43.9535,-176.5597,57.81,85,7,4.00,NZ,1685051598,waitangi,nz,
4,4,roma,43.2128,-75.4557,63.52,47,0,7.00,US,1685051396,rome,us,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "type": "hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = {"near": {"lat": latitude, "lng": longitude, "radius": radius}}
    params["bias"] = {"lat": latitude, "lng": longitude}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City_x']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
badger - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
roma - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
beau vallon - nearest hotel: No hotel found
port moresby - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
qingdao - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
nadym - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
darwin - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
palikir - national governmen

,City_ID,City_x,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country_x,Date,City_y,Country_y,Hotel Name
0,0,badger,64.8000,-147.5333,72.00,33,75,5.75,US,1685051598,badger,us,No hotel found
1,1,puerto natales,-51.7236,-72.4875,32.59,91,98,5.48,CL,1685051598,puerto natales,cl,No hotel found
2,2,yellowknife,62.4560,-114.3525,57.72,54,40,9.22,CA,1685051598,yellowknife,ca,No hotel found
3,3,waitangi,-43.9535,-176.5597,57.81,85,7,4.00,NZ,1685051598,waitangi,nz,No hotel found
4,4,roma,43.2128,-75.4557,63.52,47,0,7.00,US,1685051396,rome,us,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,568,mount gambier,-37.8333,140.7667,50.85,61,99,9.13,AU,1685051719,mount gambier,au,No hotel found
569,569,winchester,51.0651,-1.3187,57.97,67,0,5.75,GB,1685051720,winchester,gb,No hotel found
570,570,manokwari,-0.8667,134.0833,80.13,75,100,2.95,ID,1685051720,manokwari,id,No hotel found
571,571,velez rubio,37.6484,-2.0769,55.15,84,84,5.50,ES,1685051720,velez rubio,es,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='Blues',
    size='Humidity',
    title='Hotel Map',
    hover_cols=['City', 'Country', 'Hotel Name'],
    geo=True,
    tiles='CartoLight'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)